In [ ]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import spawn_camera, spawn_vehicle, preprocess_image, spawn_camera_depth

In [8]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [ ]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-1, -40, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))


In [ ]:
control = carla.VehicleControl()
control.steer = 0.0 
control.throttle = 0.5 
control.brake = 0.0  

while True:
        vehicle.apply_control(control)
        
        time.sleep(0.1)